## Import modules

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

################################### for generating synthetic data
from scipy.stats import lognorm,gamma
from scipy.optimize import brentq

################################### for neural network modeling
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

## Routines to generate synthetic data
Each function returns a sequence of event times and the average negative log-likelihood of the true model. 
Please see the main text for details of each model.

In [0]:
######################################################
### stationary poisson process
######################################################
def generate_stationary_poisson():
    tau = np.random.exponential(size=100000)
    T = tau.cumsum()
    score = 1
    return [T,score]

######################################################
### non-stationary poisson process
######################################################
def generate_nonstationary_poisson():
    L = 20000
    amp = 0.99
    l_t = lambda t: np.sin(2*np.pi*t/L)*amp + 1
    l_int = lambda t1,t2: - L/(2*np.pi)*( np.cos(2*np.pi*t2/L) - np.cos(2*np.pi*t1/L) )*amp   + (t2-t1)
    
    while 1:
        T = np.random.exponential(size=210000).cumsum()*0.5
        r = np.random.rand(210000)
        index = r < l_t(T)/2.0
        
        if index.sum() > 100000:
            T = T[index][:100000]
            score = - ( np.log(l_t(T[80000:])).sum() - l_int(T[80000-1],T[-1]) )/20000
            break
       
    return [T,score]

######################################################
### stationary renewal process
######################################################
def generate_stationary_renewal():
    s = np.sqrt(np.log(6*6+1))
    mu = -s*s/2
    tau = lognorm.rvs(s=s,scale=np.exp(mu),size=100000)
    lpdf = lognorm.logpdf(tau,s=s,scale=np.exp(mu))
    T = tau.cumsum()
    score = - np.mean(lpdf[80000:])
    
    return [T,score]
   
######################################################
### non-stationary renewal process
######################################################
def generate_nonstationary_renewal():
    L = 20000
    amp = 0.99
    l_t = lambda t: np.sin(2*np.pi*t/L)*amp + 1
    l_int = lambda t1,t2: - L/(2*np.pi)*( np.cos(2*np.pi*t2/L) - np.cos(2*np.pi*t1/L) )*amp   + (t2-t1)

    T = []
    lpdf = []
    x = 0

    k = 4
    rs = gamma.rvs(k,size=100000)
    lpdfs = gamma.logpdf(rs,k)
    rs = rs/k
    lpdfs = lpdfs + np.log(k)

    for i in range(100000):
        x_next = brentq(lambda t: l_int(x,t) - rs[i],x,x+1000)
        l = l_t(x_next)
        T.append(x_next)
        lpdf.append(  lpdfs[i] + np.log(l) )  
        x = x_next

    T = np.array(T)
    lpdf = np.array(lpdf)
    score = - lpdf[80000:].mean()
    
    return [T,score]
 
######################################################
### self-correcting process
######################################################
def generate_self_correcting():
    
    def self_correcting_process(mu,alpha,n):
    
        t = 0; x = 0;
        T = [];
        log_l = [];
        Int_l = [];
    
        for i in range(n):
            e = np.random.exponential()
            tau = np.log( e*mu/np.exp(x) + 1 )/mu # e = ( np.exp(mu*tau)- 1 )*np.exp(x) /mu
            t = t+tau
            T.append(t)
            x = x + mu*tau
            log_l.append(x)
            Int_l.append(e)
            x = x -alpha

        return [np.array(T),np.array(log_l),np.array(Int_l)]
    
    [T,log_l,Int_l] = self_correcting_process(1,1,100000)
    score = - ( log_l[80000:] - Int_l[80000:] ).sum() / 20000
    
    return [T,score]

######################################################
### Hawkes process
######################################################
def generate_hawkes1():
    [T,LL] = simulate_hawkes(100000,0.2,[0.8,0.0],[1.0,20.0])
    score = - LL[80000:].mean()
    return [T,score]

def generate_hawkes2():
    [T,LL] = simulate_hawkes(100000,0.2,[0.4,0.4],[1.0,20.0])
    score = - LL[80000:].mean()
    return [T,score]

def simulate_hawkes(n,mu,alpha,beta):
    T = []
    LL = []
    
    x = 0
    l_trg1 = 0
    l_trg2 = 0
    l_trg_Int1 = 0
    l_trg_Int2 = 0
    mu_Int = 0
    count = 0
    
    while 1:
        l = mu + l_trg1 + l_trg2
        step = np.random.exponential()/l
        x = x + step
        
        l_trg_Int1 += l_trg1 * ( 1 - np.exp(-beta[0]*step) ) / beta[0]
        l_trg_Int2 += l_trg2 * ( 1 - np.exp(-beta[1]*step) ) / beta[1]
        mu_Int += mu * step
        l_trg1 *= np.exp(-beta[0]*step)
        l_trg2 *= np.exp(-beta[1]*step)
        l_next = mu + l_trg1 + l_trg2
        
        if np.random.rand() < l_next/l: #accept
            T.append(x)
            LL.append( np.log(l_next) - l_trg_Int1 - l_trg_Int2 - mu_Int )
            l_trg1 += alpha[0]*beta[0]
            l_trg2 += alpha[1]*beta[1]
            l_trg_Int1 = 0
            l_trg_Int2 = 0
            mu_Int = 0
            count += 1
            
            if count == n:
                break
        
    return [np.array(T),np.array(LL)]

## Routines for neural network modeling

In [0]:
######################################################
### constant hazard function
######################################################
class HAZARD_const():
    
    class Layer_LL(layers.Layer):

        def __init__(self, **kwargs):
            super(HAZARD_const.Layer_LL, self).__init__(**kwargs)

        def build(self, input_shape):
            self.build = True

        def call(self, inputs):
            x = inputs[0]; p = inputs[1];
            log_l = p
            Int_l = K.exp( p ) * x
            return [log_l,Int_l]

        def compute_output_shape(self, input_shape):
            return [input_shape[0],input_shape[0]]
               
    def __call__(self,inputs):
        x = inputs[0]; rnn = inputs[1];
        p = layers.Dense(1)(rnn)
        [log_l,Int_l] = HAZARD_const.Layer_LL()([x,p])
        LL = layers.Subtract()([log_l,Int_l])
        return [LL,log_l,Int_l]
    
######################################################
### exponential hazard function
######################################################
class HAZARD_exp():
    
    class Layer_LL(layers.Layer):

        def __init__(self, **kwargs):
            super(HAZARD_exp.Layer_LL, self).__init__(**kwargs)

        def build(self, input_shape):
            self.a = self.add_weight(name='a', initializer= keras.initializers.Constant(value=1.0), shape=(), trainable=True)
            self.build = True

        def call(self, inputs):
            x = inputs[0]; p = inputs[1];
            a = self.a;
            log_l = p - a*x
            Int_l = K.exp( p ) * ( 1 - K.exp(-a*x) ) / a
            return [log_l,Int_l]

        def compute_output_shape(self, input_shape):
            return [input_shape[0],input_shape[0]]
               
    def __call__(self,inputs):
        x = inputs[0]; rnn = inputs[1];
        p = layers.Dense(1)(rnn)
        [log_l,Int_l] = HAZARD_exp.Layer_LL()([x,p])
        LL = layers.Subtract()([log_l,Int_l])
        return [LL,log_l,Int_l]

######################################################
### piecewise constant hazard function
######################################################   
class HAZARD_pc():
    
    class Layer_LL(layers.Layer):

        def __init__(self, size_div,t_max,**kwargs):
            self.size_div = size_div
            self.t_max = t_max
            self.bin_l = K.constant(np.linspace(0,t_max,size_div+1)[:-1].reshape(1,-1))
            self.bin_r = K.constant(np.linspace(0,t_max,size_div+1)[1:].reshape(1,-1))
            self.width = K.constant(t_max/size_div)
            self.ones = K.constant(np.ones([size_div,1]))
            super(HAZARD_pc.Layer_LL, self).__init__(**kwargs)

        def build(self, input_shape):
            self.build = True

        def call(self, inputs):
            x = inputs[0]; p = inputs[1];

            r_le = K.cast( K.greater_equal(x,self.bin_l), dtype=K.floatx() ) 
            r_er = K.cast( K.less(         x,self.bin_r), dtype=K.floatx() )
            r_e  = r_er*r_le
            r_l  = 1-r_er
            
            log_l = K.log(K.dot(p*r_e,self.ones))
            Int_l = K.dot(p*r_l*self.width,self.ones) + K.dot(p*(x-self.bin_l)*r_e,self.ones) 

            return [log_l,Int_l]

        def compute_output(self,input_shape):
            return [input_shape[0],input_shape[0],input_shape[0]]

    def __init__(self,size_div,t_max):
        self.size_div = size_div
        self.t_max = t_max
        
    def __call__(self,inputs):
        x = inputs[0]; rnn = inputs[1];
        p = layers.Dense(self.size_div,activation='softplus')(rnn)
        [log_l,Int_l] = HAZARD_pc.Layer_LL(self.size_div,self.t_max)([x,p])
        LL = layers.Subtract()([log_l,Int_l])
        return [LL,log_l,Int_l]
    
        
######################################################
### neural network based hazard function
######################################################
class HAZARD_NN():
    
    def __init__(self, size_layer, size_nn, log_mode=True):
        self.size_layer = size_layer
        self.size_nn = size_nn
        self.log_mode = log_mode
          
    def __call__(self,inputs):
        x = inputs[0]; rnn = inputs[1];
        
        if self.log_mode:
            x_nmlz = layers.Lambda(lambda x: (K.log(x)-self.mu_x)/self.sigma_x )(x) 
        else:
            x_nmlz = layers.Lambda(lambda x: (x-self.mu_x)/self.sigma_x )(x) 
            
        def abs_glorot_uniform(shape, dtype=None, partition_info=None):
            return K.abs(keras.initializers.glorot_uniform(seed=None)(shape,dtype=dtype))
        
        hidden_x = layers.Dense(self.size_nn,kernel_initializer=abs_glorot_uniform,kernel_constraint=keras.constraints.NonNeg(),use_bias=False)(x_nmlz)
        hidden_p = layers.Dense(self.size_nn)(rnn)
        hidden = layers.Add()([hidden_x,hidden_p])
        hidden = layers.Activation('tanh')(hidden)
        
        for i in range(self.size_layer-1):
            hidden = layers.Dense(self.size_nn,activation='tanh',kernel_initializer=abs_glorot_uniform,kernel_constraint=keras.constraints.NonNeg())(hidden)
            
        Int_l = layers.Dense(1, activation='softplus',kernel_initializer=abs_glorot_uniform, kernel_constraint=keras.constraints.NonNeg() )(hidden)
        log_l = layers.Lambda(lambda inputs: K.log( 1e-10 + K.gradients(inputs[0],inputs[1])[0] ))([Int_l,x])
        LL = layers.Subtract()([log_l,Int_l])
        
        return [LL,log_l,Int_l]
        
    def normalize_input(self,x):
        
        if self.log_mode:
            self.mu_x = np.log(x).mean() 
            self.sigma_x = np.log(x).std()   
        else:
            self.mu_x = x.mean()
            self.sigma_x = x.std()
        
        return self
    

######################################################
### RNN
######################################################
class RNN_PP():
    
    def __init__(self, size_rnn, time_step, log_mode=True):
        self.size_rnn= size_rnn
        self.time_step = time_step
        self.log_mode = log_mode
        
    def __call__(self,inputs):
        x = inputs
        
        if self.log_mode:
            x_nmlz = layers.Lambda(lambda x: (K.log(x)-self.mu_x)/self.sigma_x )(x) 
        else:
            x_nmlz = layers.Lambda(lambda x: (x-self.mu_x)/self.sigma_x )(x) 
            
        rnn = layers.SimpleRNN(self.size_rnn,input_shape=(self.time_step,1),activation='tanh')(x_nmlz)
        
        return rnn
    
    def normalize_input(self,x):
        
        if self.log_mode:
            self.mu_x = np.log(x).mean() 
            self.sigma_x = np.log(x).std()   
        else:
            self.mu_x = x.mean()
            self.sigma_x = x.std()
        
        return self
    
    
######################################################################################
### a class for a recurrent neural network based model for temporal point processes
######################################################################################
class NPP():
    
    def __init__(self,time_step,size_rnn,type_hazard,size_layer=2,size_nn=64,size_div=128,log_mode=True):
        self.time_step = time_step
        self.size_rnn = size_rnn
        self.type_hazard = type_hazard
        self.size_layer = size_layer
        self.size_nn = size_nn
        self.log_mode = log_mode
        self.size_div = size_div
            
    def set_data(self,T):
        
        def rolling_matrix(x,window_size):
            x = x.flatten()
            n = x.shape[0]
            stride = x.strides[0]
            return np.lib.stride_tricks.as_strided(x, shape=(n-window_size+1, window_size), strides=(stride,stride) ).copy()

        def transform_data(T,n_train,n_test,time_step):

            np.random.seed(0)

            index_shuffle = np.random.permutation(n_train-time_step-1)

            dT_train = np.ediff1d(T[:n_train])
            r_dT_train = rolling_matrix(dT_train,time_step+1)[index_shuffle]

            dT_test = np.ediff1d(T[n_train-time_step-1:n_train+n_test])
            r_dT_test = rolling_matrix(dT_test,time_step+1)

            dT_train_input  = r_dT_train[:,:-1].reshape(-1,time_step,1)
            dT_train_target = r_dT_train[:,[-1]]
            dT_test_input  = r_dT_test[:,:-1].reshape(-1,time_step,1)
            dT_test_target = r_dT_test[:,[-1]]

            return [dT_train_input,dT_train_target,dT_test_input,dT_test_target]
        
        n = T.shape[0]       
        [dT_train_input,dT_train_target,dT_test_input,dT_test_target] = transform_data(T,int(n*0.8),n-int(n*0.8),self.time_step) # A sequence is divided into training and test data.
            
        self.dT_train_input  = dT_train_input
        self.dT_train_target = dT_train_target
        self.dT_test_input   = dT_test_input
        self.dT_test_target  = dT_test_target

        self.n_train = dT_train_target.shape[0]
        self.n_test  = dT_test_target.shape[0]
        
        self.t_max = np.max(np.vstack([self.dT_train_target,self.dT_test_target]))*1.001
        
        #print("n_train: %d, n_test: %d, t_max: %f" % (self.n_train,self.n_test,self.t_max) )
       
        return self
        
    def set_model(self):
        
        if self.type_hazard == 'const':
            self.layer_hazard = HAZARD_const()
        elif self.type_hazard == 'exp':
            self.layer_hazard = HAZARD_exp()
        elif self.type_hazard == 'pc':
            self.layer_hazard = HAZARD_pc(size_div=self.size_div,t_max=self.t_max)
        elif self.type_hazard == 'NN':
            self.layer_hazard = HAZARD_NN(size_layer=self.size_layer,size_nn=self.size_nn,log_mode=self.log_mode).normalize_input(self.dT_train_target)
        
        self.rnn = RNN_PP(size_rnn=self.size_rnn,time_step=self.time_step,log_mode=self.log_mode).normalize_input(self.dT_train_target)
        
        input_dT = layers.Input(shape=(self.time_step,1))
        input_x = layers.Input(shape=(1,))
        output_rnn = self.rnn(input_dT)
        [LL,log_l,Int_l] = self.layer_hazard([input_x,output_rnn])
        self.model = Model(inputs=[input_dT,input_x],outputs=[LL,log_l,Int_l])
        self.model.add_loss(-K.mean(LL))
        #model.summary()      
        
        return self
    
    def compile(self,lr=1e-3):
        self.model.compile(keras.optimizers.Adam(lr=lr))
        return self
    
    def scores(self):
        scores = - self.model.predict([self.dT_test_input,self.dT_test_target],batch_size=self.n_test)[0].flatten()
        return scores
    
    class CustomEarlyStopping(keras.callbacks.Callback):
    
        def __init__(self):
            super(NPP.CustomEarlyStopping, self).__init__()
            self.best_val_loss = 100000
            self.history_val_loss = []
            self.best_weights   = None

        def on_epoch_end(self, epoch, logs=None):
            
            val_loss = logs['val_loss']
            self.history_val_loss = np.append(self.history_val_loss,val_loss)

            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.best_weights = self.model.get_weights()
                
            if self.best_val_loss + 0.05 < val_loss: 
                    self.model.stop_training = True
                
            if (epoch+1) % 5 == 0:
                
                #print('epoch: %d, current_val_loss: %f, min_val_loss: %f' % (epoch+1,val_loss,self.best_val_loss) )
                
                if (epoch+1) >= 15:
                    if self.best_val_loss > self.history_val_loss[:-5].min() - 0.001: 
                        self.model.stop_training = True
                        
        def on_train_end(self,logs=None):
            self.model.set_weights(self.best_weights)
            #print('set optimal weights')
        
    def fit_eval(self,epochs=100,batch_size=256):
        
        es = NPP.CustomEarlyStopping()
        history = self.model.fit([self.dT_train_input,self.dT_train_target],epochs=epochs,batch_size=batch_size,validation_split=0.2,verbose=0,callbacks=[es])
        scores = self.scores()
        
        self.history = {'loss':np.array(history.history['loss']), 'val_loss':np.array(history.history['val_loss'])}
        self.val_loss = es.best_val_loss
        self.mnll = scores.mean()
        
        
        self.mae = self.mean_absolute_error()
        
        #print('score: %f' % scores.mean() )
        #print()
            
        return self
    
    def bisect_target(self,taus): 
        return self.model.predict([self.dT_test_input,taus],batch_size=taus.shape[0])[2] - np.log(2)

    def median_prediction(self,l,r):

        for i in range(13):
            c = (l+r)/2
            v = self.bisect_target(c)
            l = np.where(v<0,c,l)
            r = np.where(v>=0,c,r)
          
        return (l+r)/2
        
    def mean_absolute_error(self):
        l=np.mean(self.dT_train_target)*0.0001*np.ones_like(self.dT_test_target)
        r=np.mean(self.dT_train_target)*100.0  *np.ones_like(self.dT_test_target)
        tau_pred = self.median_prediction(l,r) 
        return np.mean(np.abs(tau_pred-self.dT_test_target))


class training():

    def training(self,type_hazard,T):
        min_val_loss = 100000
        for time_step in [5,10,20,40]:
            npp = NPP(time_step=time_step,type_hazard=type_hazard,size_rnn=64,size_layer=2,size_nn=64,size_div=128,log_mode=True).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)
            if npp.val_loss < min_val_loss:
                min_val_loss = npp.val_loss
                self.mnll = npp.mnll
                self.mae = npp.mae

        return self

        

## Demo1
A python class "NPP" splits the data into training and test data, trains the model using the training data, and evaluates the model using the test data.

Note that we fix the values of the hyper-parameters in this section for simplicity.

In [0]:
########## Hyper-parameters
time_step = 20 # truncation depth of a RNN 
size_rnn = 64  # the number of units in a RNN
size_div = 128 # the number of sub-intervals of the piecewise constant function (piecewise constant model)
size_nn = 64   # the number of units in each hidden layer of the cumulative hazard function network (neural network based model)
size_layer = 2 # the number of hidden layers of the cumulative hazard function network (neural network based model)

########## Generate data
[T,score_ref] = generate_stationary_poisson()     # generate synthetic data: stationary Poisson process.
print('#######################################')
print('## Stationary Poisson process')
print('#######################################')
print()

########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='const').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# exponential model
npp2 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='exp').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# piecewise constant model
npp3 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='pc',size_div=size_div).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# neural network based model
npp4 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='NN',size_layer=size_layer,size_nn=size_nn).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Stationary Poisson process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.981 (standardized score: -0.019), MAE: 0.679
## exponential model          
     MNLL: 0.981 (standardized score: -0.019), MAE: 0.678
## piecewise constant model   
     MNLL: 0.984 (standardized score: -0.016), MAE: 0.679
## neural network based model 
     MNLL: 0.982 (standardized score: -0.018), MAE: 0.679


In [0]:
########## Hyper-parameters
time_step = 20 # truncation depth of a RNN 
size_rnn = 64  # the number of units in a RNN
size_div = 128 # the number of sub-intervals of the piecewise constant function (piecewise constant model)
size_nn = 64   # the number of units in each hidden layer of the cumulative hazard function network (neural network based model)
size_layer = 2 # the number of hidden layers of the cumulative hazard function network (neural network based model)

########## Generate data
[T,score_ref] = generate_nonstationary_poisson() # generate synthetic data: nonstationary Poisson process.
print('#######################################')
print('## Non-stationary Poisson process')
print('#######################################')
print()


########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='const').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# exponential model
npp2 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='exp').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# piecewise constant model
npp3 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='pc',size_div=size_div).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# neural network based model
npp4 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='NN',size_layer=size_layer,size_nn=size_nn).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )

#######################################
## Non-stationary Poisson process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.724 (standardized score: 0.021), MAE: 0.701
## exponential model          
     MNLL: 0.723 (standardized score: 0.021), MAE: 0.710
## piecewise constant model   
     MNLL: 0.726 (standardized score: 0.023), MAE: 0.715
## neural network based model 
     MNLL: 0.722 (standardized score: 0.019), MAE: 0.702


In [0]:
########## Hyper-parameters
time_step = 20 # truncation depth of a RNN 
size_rnn = 64  # the number of units in a RNN
size_div = 128 # the number of sub-intervals of the piecewise constant function (piecewise constant model)
size_nn = 64   # the number of units in each hidden layer of the cumulative hazard function network (neural network based model)
size_layer = 2 # the number of hidden layers of the cumulative hazard function network (neural network based model)

########## Generate data
[T,score_ref] = generate_stationary_renewal()    # generate synthetic data: stationary renewal process.
print('#######################################')
print('## Stationary Renewal process')
print('#######################################')
print()


########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='const').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# exponential model
npp2 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='exp').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# piecewise constant model
npp3 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='pc',size_div=size_div).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# neural network based model
npp4 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='NN',size_layer=size_layer,size_nn=size_nn).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Stationary Renewal process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 1.033 (standardized score: 0.766), MAE: 1.164
## exponential model          
     MNLL: 0.565 (standardized score: 0.298), MAE: 0.996
## piecewise constant model   
     MNLL: 0.840 (standardized score: 0.574), MAE: 1.070
## neural network based model 
     MNLL: 0.268 (standardized score: 0.002), MAE: 0.972


In [0]:
########## Hyper-parameters
time_step = 20 # truncation depth of a RNN 
size_rnn = 64  # the number of units in a RNN
size_div = 128 # the number of sub-intervals of the piecewise constant function (piecewise constant model)
size_nn = 64   # the number of units in each hidden layer of the cumulative hazard function network (neural network based model)
size_layer = 2 # the number of hidden layers of the cumulative hazard function network (neural network based model)

########## Generate data
[T,score_ref] = generate_nonstationary_renewal() # generate synthetic data: nonstationary renewal process.
print('#######################################')
print('## Non-tationary Renewal process')
print('#######################################')
print()


########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='const').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# exponential model
npp2 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='exp').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# piecewise constant model
npp3 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='pc',size_div=size_div).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# neural network based model
npp4 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='NN',size_layer=size_layer,size_nn=size_nn).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )

#######################################
## Non-tationary Renewal process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.708 (standardized score: 0.368), MAE: 0.441
## exponential model          
     MNLL: 0.695 (standardized score: 0.355), MAE: 0.436
## piecewise constant model   
     MNLL: 0.676 (standardized score: 0.336), MAE: 0.436
## neural network based model 
     MNLL: 0.367 (standardized score: 0.027), MAE: 0.403


In [0]:
########## Hyper-parameters
time_step = 20 # truncation depth of a RNN 
size_rnn = 64  # the number of units in a RNN
size_div = 128 # the number of sub-intervals of the piecewise constant function (piecewise constant model)
size_nn = 64   # the number of units in each hidden layer of the cumulative hazard function network (neural network based model)
size_layer = 2 # the number of hidden layers of the cumulative hazard function network (neural network based model)

########## Generate data
[T,score_ref] = generate_self_correcting()       # generate synthetic data: self-correcting process.
print('#######################################')
print('## Self-correcting process')
print('#######################################')
print()


########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='const').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# exponential model
npp2 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='exp').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# piecewise constant model
npp3 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='pc',size_div=size_div).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# neural network based model
npp4 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='NN',size_layer=size_layer,size_nn=size_nn).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )

#######################################
## Self-correcting process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.940 (standardized score: 0.180), MAE: 0.532
## exponential model          
     MNLL: 0.788 (standardized score: 0.029), MAE: 0.498
## piecewise constant model   
     MNLL: 0.798 (standardized score: 0.038), MAE: 0.498
## neural network based model 
     MNLL: 0.791 (standardized score: 0.032), MAE: 0.499


In [0]:
########## Hyper-parameters
time_step = 20 # truncation depth of a RNN 
size_rnn = 64  # the number of units in a RNN
size_div = 128 # the number of sub-intervals of the piecewise constant function (piecewise constant model)
size_nn = 64   # the number of units in each hidden layer of the cumulative hazard function network (neural network based model)
size_layer = 2 # the number of hidden layers of the cumulative hazard function network (neural network based model)

########## Generate data
[T,score_ref] = generate_hawkes1()               # generate synthetic data: hawkes1 process.
print('#######################################')
print('## Hawkes1 process')
print('#######################################')
print()


########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='const').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# exponential model
npp2 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='exp').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# piecewise constant model
npp3 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='pc',size_div=size_div).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# neural network based model
npp4 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='NN',size_layer=size_layer,size_nn=size_nn).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )

#######################################
## Hawkes1 process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.673 (standardized score: 0.213), MAE: 0.922
## exponential model          
     MNLL: 0.533 (standardized score: 0.073), MAE: 0.842
## piecewise constant model   
     MNLL: 0.473 (standardized score: 0.013), MAE: 0.837
## neural network based model 
     MNLL: 0.480 (standardized score: 0.020), MAE: 0.837


In [0]:
########## Hyper-parameters
time_step = 20 # truncation depth of a RNN 
size_rnn = 64  # the number of units in a RNN
size_div = 128 # the number of sub-intervals of the piecewise constant function (piecewise constant model)
size_nn = 64   # the number of units in each hidden layer of the cumulative hazard function network (neural network based model)
size_layer = 2 # the number of hidden layers of the cumulative hazard function network (neural network based model)

########## Generate data
[T,score_ref] = generate_hawkes2() # generate synthetic data: hawkes2 process.
print('#######################################')
print('## Hawkes2 process')
print('#######################################')
print()


########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='const').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# exponential model
npp2 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='exp').set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# piecewise constant model
npp3 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='pc',size_div=size_div).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

# neural network based model
npp4 = NPP(time_step=time_step,size_rnn=size_rnn,type_hazard='NN',size_layer=size_layer,size_nn=size_nn).set_data(T).set_model().compile(lr=0.001).fit_eval(batch_size=256)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )

#######################################
## Hawkes2 process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.672 (standardized score: 0.671), MAE: 1.132
## exponential model          
     MNLL: 0.414 (standardized score: 0.413), MAE: 1.017
## piecewise constant model   
     MNLL: 0.206 (standardized score: 0.205), MAE: 1.003
## neural network based model 
     MNLL: 0.019 (standardized score: 0.018), MAE: 0.999


# Demo2 (take a bit of time)

A python class "training" optimizes *time_steps* with the cross validation. 

In [0]:
########## Generate data
[T,score_ref] = generate_stationary_poisson()     # generate synthetic data: stationary Poisson process.
print('#######################################')
print('## Stationary Poisson process')
print('#######################################')
print()

########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = training().training('const',T)

# exponential model
npp2 = training().training('exp',T)

# piecewise constant model
npp3 = training().training('pc',T)

# neural network based model
npp4 = training().training('NN',T)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Stationary Poisson process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 1.009 (standardized score: 0.009), MAE: 0.703
## exponential model          
     MNLL: 1.009 (standardized score: 0.009), MAE: 0.702
## piecewise constant model   
     MNLL: 1.012 (standardized score: 0.012), MAE: 0.703
## neural network based model 
     MNLL: 1.011 (standardized score: 0.011), MAE: 0.703


In [0]:
########## Generate data
[T,score_ref] = generate_nonstationary_poisson() # generate synthetic data: nonstationary Poisson process.
print('#######################################')
print('## Non-stationary Poisson process')
print('#######################################')
print()

########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = training().training('const',T)

# exponential model
npp2 = training().training('exp',T)

# piecewise constant model
npp3 = training().training('pc',T)

# neural network based model
npp4 = training().training('NN',T)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Non-stationary Poisson process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.719 (standardized score: 0.016), MAE: 0.710
## exponential model          
     MNLL: 0.719 (standardized score: 0.016), MAE: 0.713
## piecewise constant model   
     MNLL: 0.718 (standardized score: 0.015), MAE: 0.711
## neural network based model 
     MNLL: 0.732 (standardized score: 0.030), MAE: 0.710


In [0]:
########## Generate data
[T,score_ref] = generate_stationary_renewal()    # generate synthetic data: stationary renewal process.
print('#######################################')
print('## Stationary Renewal process')
print('#######################################')
print()

########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = training().training('const',T)

# exponential model
npp2 = training().training('exp',T)

# piecewise constant model
npp3 = training().training('pc',T)

# neural network based model
npp4 = training().training('NN',T)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Stationary Renewal process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.992 (standardized score: 0.737), MAE: 1.125
## exponential model          
     MNLL: 0.544 (standardized score: 0.289), MAE: 0.958
## piecewise constant model   
     MNLL: 0.903 (standardized score: 0.648), MAE: 1.077
## neural network based model 
     MNLL: 0.256 (standardized score: 0.001), MAE: 0.932


In [0]:
########## Generate data
[T,score_ref] = generate_nonstationary_renewal()    # generate synthetic data: nonstationary renewal process.
print('#######################################')
print('## Non-stationary Renewal process')
print('#######################################')
print()

########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = training().training('const',T)

# exponential model
npp2 = training().training('exp',T)

# piecewise constant model
npp3 = training().training('pc',T)

# neural network based model
npp4 = training().training('NN',T)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Non-stationary Renewal process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.705 (standardized score: 0.375), MAE: 0.431
## exponential model          
     MNLL: 0.699 (standardized score: 0.369), MAE: 0.436
## piecewise constant model   
     MNLL: 0.683 (standardized score: 0.353), MAE: 0.422
## neural network based model 
     MNLL: 0.352 (standardized score: 0.021), MAE: 0.392


In [0]:
########## Generate data
[T,score_ref] = generate_self_correcting()     # generate synthetic data: self-correcting process.
print('#######################################')
print('## Self-correcting process')
print('#######################################')
print()

########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = training().training('const',T)

# exponential model
npp2 = training().training('exp',T)

# piecewise constant model
npp3 = training().training('pc',T)

# neural network based model
npp4 = training().training('NN',T)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Self-correcting process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.936 (standardized score: 0.175), MAE: 0.538
## exponential model          
     MNLL: 0.789 (standardized score: 0.028), MAE: 0.499
## piecewise constant model   
     MNLL: 0.803 (standardized score: 0.042), MAE: 0.502
## neural network based model 
     MNLL: 0.796 (standardized score: 0.035), MAE: 0.502


In [0]:
########## Generate data
[T,score_ref] = generate_hawkes1() # generate synthetic data: hawkes1 process.
print('#######################################')
print('## Hawkes1 process')
print('#######################################')
print()

########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = training().training('const',T)

# exponential model
npp2 = training().training('exp',T)

# piecewise constant model
npp3 = training().training('pc',T)

# neural network based model
npp4 = training().training('NN',T)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Hawkes1 process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.762 (standardized score: 0.234), MAE: 0.989
## exponential model          
     MNLL: 0.610 (standardized score: 0.082), MAE: 0.918
## piecewise constant model   
     MNLL: 0.544 (standardized score: 0.016), MAE: 0.915
## neural network based model 
     MNLL: 0.537 (standardized score: 0.009), MAE: 0.913


In [0]:
########## Generate data
[T,score_ref] = generate_hawkes2() # generate synthetic data: hawkes2 process.
print('#######################################')
print('## Hawkes2 process')
print('#######################################')
print()

########## Train and evaluate the model (The following code will raise warnings, but please ignore them.)

# constant model
npp1 = training().training('const',T)

# exponential model
npp2 = training().training('exp',T)

# piecewise constant model
npp3 = training().training('pc',T)

# neural network based model
npp4 = training().training('NN',T)

print()
print('#######################################')
print('## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)')
print('#######################################')
print('## constand model             \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp1.mnll,npp1.mnll-score_ref,npp1.mae) )
print('## exponential model          \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp2.mnll,npp2.mnll-score_ref,npp2.mae) )
print('## piecewise constant model   \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp3.mnll,npp3.mnll-score_ref,npp3.mae) )
print('## neural network based model \n     MNLL: %.3f (standardized score: %.3f), MAE: %.3f' % (npp4.mnll,npp4.mnll-score_ref,npp4.mae) )


#######################################
## Hawkes2 process
#######################################


#######################################
## Performance for test data (MNLL: mean negative log-likelihood, MAE: mean absolute error)
#######################################
## constand model             
     MNLL: 0.701 (standardized score: 0.654), MAE: 1.135
## exponential model          
     MNLL: 0.438 (standardized score: 0.392), MAE: 1.010
## piecewise constant model   
     MNLL: 0.237 (standardized score: 0.190), MAE: 0.997
## neural network based model 
     MNLL: 0.061 (standardized score: 0.015), MAE: 0.993
